In [1]:
import os
from os.path import join, exists, dirname, basename
from glob import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
with open('/home/zacharykeskinen/Documents/infrasound/array_data/merged/full_days', 'rb') as f:
    days = pickle.load(f)

In [3]:
for day, fps in days.items():
    if day == '2022-02-02':
        for h, f in fps.items():
            df = pd.read_parquet(f)['pa']
            #plt.plot(arr[::100])

In [6]:
df.plot()